In [1]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack import Pipeline
from haystack.components.routers import ConditionalRouter
import xml.etree.ElementTree as ET
import boto3
import json

import os
from getpass import getpass
from haystack_integrations.components.generators.amazon_bedrock import AmazonBedrockGenerator


from datasets import load_dataset
from haystack import Document
from inputs import matrix, operational_management_skillset_explanation, NOTES_PER_EXERCISE

In [2]:
def generate_skill_xml_string(skill_name, skill_value, skill_description):
    root = ET.Element("Skill")
    name = ET.SubElement(root, "Name")
    name.text = skill_name
    value = ET.SubElement(root, "Value")
    value.text = str(skill_value)    
    description = ET.SubElement(root, "Description")
    for item in skill_description:
        point = ET.SubElement(description, "Point")
        point.text = item
    xml_str = ET.tostring(root, encoding='unicode', method='xml')
    return xml_str

In [3]:
notes = ""
for e in NOTES_PER_EXERCISE:
    notes += e + "\n"
    notes += NOTES_PER_EXERCISE[e]

In [4]:
template = """You are an expert HR consultant specializing in evaluating human psychology and behavior during candidate assessments. Your task is to analyze the alignment between qualitative observation notes and a quantitative skill evaluation for a job candidate.

Context:
- Observation Notes: Detailed qualitative observations from the assessment process
- Numerical Skill Evaluation: A rating of the candidate's skills on a scale from 0 to 4
  0 = Not observed/Not applicable
  1 = Poor performance
  2 = Good/Expected performance
  3 = Very good performance
  4 = Excellent performance
- Skill Description: Detailed explanation of the skill's attributes and expectations

Instructions:
1. Carefully read the observation notes and skill description.
2. For each attribute of the skill:
   a. Identify specific notes that support or contradict the numerical rating.
   b. Evaluate the strength and relevance of each piece of evidence.
   c. Provide supporting and opposing evidence for every point of the skill, even if one type of evidence is weaker or less prevalent.
3. Determine if the overall observations align with the provided numerical evaluation.
4. Provide your analysis in the structured format below.

Output Format:

Skill Name: [Name of the skill being evaluated]
Numerical Rating: [The provided numerical rating (0-4)]

Attribute Analysis:

1. [Specific attribute of the skill]

Supporting Evidence:
- "[Exact quote from observation notes]"
  Explanation: [Why this supports the rating]
- "[Another supporting quote]"
  Explanation: [Why this supports the rating]

Opposing Evidence:
- "[Exact quote from observation notes]"
  Explanation: [Why this contradicts the rating]
- "[Another opposing quote]"
  Explanation: [Why this contradicts the rating]

Attribute Analysis: [You assessment whether the attribute is satisfied or not]

2. [Next attribute of the skill]

[Repeat the above structure for each attribute]

Overall Analysis:
[Your overall assessment of whether the observations align with the numerical rating, including any nuances or potential discrepancies across all attributes]

Evidence to evaluation matching score: [High/Medium/Low - Your level of confidence if based on your analysis the evidence and the numerical value match]

Suggested score: [Score you would give]

Evidence Summary:
Total Supporting Points: [Number of supporting points across all attributes]
Total Opposing Points: [Number of opposing points across all attributes]
Total Attributes confirmed: [Number of attributes that have good allignment with the numerical rating]
Total Attributes disproved: [Number of attributes that have a bad allignment with the numerical rating]
Net Evidence Score: [Total Supporting Points - Total Opposing Points]

Important:
- Be objective and thorough in your analysis.
- Consider the context and specific requirements of the skill.
- If there's insufficient evidence to make a determination, state this clearly by using NONE label

Observation Notes:
{observation_notes}

Skill Description and Numerical Rating:
{skill}

Please provide your analysis based on the instructions and format above.
"""

In [5]:
client = boto3.client('bedrock-runtime')

In [6]:

def get_prompt_results(skill_name):

    skill_value = matrix[skill_name]
    skill_description = operational_management_skillset_explanation[skill_name]

    skill = generate_skill_xml_string(skill_name, skill_value, skill_description)

    final_prompt = template.format(observation_notes=notes, skill=skill)

    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    
    # Format the request payload using the model's native structure.
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 8000,
        "temperature": 0.5,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": final_prompt}],
            }
        ],
    }
    request = json.dumps(native_request)

    try:
        # Invoke the model with the request.
        response = client.invoke_model(modelId=model_id, body=request)
    
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)
    
    # Decode the response body.
    model_response = json.loads(response["body"].read())

    response_text = model_response["content"][0]["text"]
    return response_text

In [7]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor() as executor:
    # Launch all the processes in parallel and store their future objects
    futures = [executor.submit(get_prompt_results, skill_name) for skill_name in matrix]
    
    # Retrieve the results as they complete
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

print(results)

['Skill Name: Kunden verstehen\nNumerical Rating: 3.5\n\nAttribute Analysis:\n\n1. Informiert sich über Trends und Entwicklungen der Kunden.\n\nSupporting Evidence:\n- "Vermittelt einen gesamtheitlichen Blick über Prozesse und Strukturen in seinem Verantwortungsbereich; verfolgt das intrinsische Ziel, diese Prozesse hinsichtlich der Effizienz am Leben zu erhalten sowie den Puls des Zusammenarbeitsgeflechts zu erspüren"\n  Explanation: This indicates that the candidate is aware of and actively monitors trends and developments in their area of responsibility, which likely includes customer-related processes.\n\nOpposing Evidence:\nNONE\n\nAttribute Analysis: This attribute is satisfied based on the available evidence.\n\n2. Optimiert Arbeitsweisen im eigenen Verantwortungsbereich, um noch bessere Ergebnisse für Kunden erbringen zu können.\n\nSupporting Evidence:\n- "Beweist im Arbeitsstil eine hohe Strukturorientierung, logisches Vorgehen sowie eine fokussierte Ergebnisgenerierung"\n  Ex

In [8]:
for i in results:
    print('***************************************')
    print(i)

***************************************
Skill Name: Kunden verstehen
Numerical Rating: 3.5

Attribute Analysis:

1. Informiert sich über Trends und Entwicklungen der Kunden.

Supporting Evidence:
- "Vermittelt einen gesamtheitlichen Blick über Prozesse und Strukturen in seinem Verantwortungsbereich; verfolgt das intrinsische Ziel, diese Prozesse hinsichtlich der Effizienz am Leben zu erhalten sowie den Puls des Zusammenarbeitsgeflechts zu erspüren"
  Explanation: This indicates that the candidate is aware of and actively monitors trends and developments in their area of responsibility, which likely includes customer-related processes.

Opposing Evidence:
NONE

Attribute Analysis: This attribute is satisfied based on the available evidence.

2. Optimiert Arbeitsweisen im eigenen Verantwortungsbereich, um noch bessere Ergebnisse für Kunden erbringen zu können.

Supporting Evidence:
- "Beweist im Arbeitsstil eine hohe Strukturorientierung, logisches Vorgehen sowie eine fokussierte Ergebni